<H1 align="center"> 🏒 Project: NHL 🏒 </H1>

Data for this project can be found in [this](https://www.kaggle.com/datasets/martinellis/nhl-game-data) kaggle link. According to the author:

> The data represents all the official metrics measured for each game in the NHL (from 200 to 2019). I intend to update it semi-regularly depending on development progress of my database server.

We loaded the data into a SQLite databse using the simple [DB Browser for SQLite](https://sqlitebrowser.org/).

## Data exploration with SQL

Using our SQLite database we will answer exploratory questions using SQL queries. To this end, we'll use python's `sqlite3` module to run queries and `pandas` to get our results in dataframe objects.

The schema for this database is given in the figure bellow

![DB Schema](table_relationships.JPG)

In [1]:
import sqlite3 as sql
import pandas as pd

# Connecting to Database

In [2]:
con = sql.connect("nhl-data.db")
cur = con.cursor()

# Total number of goals (and games) per season

In [3]:
# Number of goals and games per season
query = """
SELECT
    SUBSTR(season, 1, 4) AS season_start,
    SUM(away_goals) + SUM(home_goals) AS total_goals,
    COUNT(game_id) AS number_of_games,
    CAST(SUM(away_goals) + SUM(home_goals) AS REAL) / COUNT(game_id) AS goals_per_game
FROM
    game
GROUP BY
    season_start
ORDER BY
    season_start;
"""
df_goals_season = pd.read_sql(query, con)

In [4]:
df_goals_season

,season_start,total_goals,number_of_games,goals_per_game
0,2000,6782,1230,5.513821
1,2001,6442,1230,5.237398
2,2002,6530,1230,5.308943
3,2003,6318,1230,5.136585
4,2005,7588,1230,6.169106
5,2006,7246,1230,5.891057
6,2007,6847,1230,5.566667
7,2008,7165,1230,5.825203
8,2009,6987,1230,5.680488
9,2010,7369,1319,5.586808


In [5]:
# df_goals_season.to_markdown()

## Goals per team per season

In [6]:
query = """
SELECT
	SUBSTR(game_plays.game_id, 1, 4) AS season,
	team_info.teamName,
	COUNT(game_plays.event) AS number_of_goals,
	game_plays.team_id_for	
FROM
	game_plays
INNER JOIN
	team_info
		ON game_plays.team_id_for = team_info.team_id
WHERE
	event = 'Goal'
GROUP BY
	season, game_plays.team_id_for
ORDER BY
	season, number_of_goals DESC;
"""
df_teams_season = pd.read_sql(query, con)

In [7]:
df_teams_season.head()

,season,teamName,number_of_goals,team_id_for
0,2000,Devils,293,1
1,2000,Penguins,271,5
2,2000,Avalanche,263,21
3,2000,Senators,258,9
4,2000,Red Wings,253,17


Filtering a specific team:

In [8]:
df_teams_season[df_teams_season['teamName'] == 'Flames']

,season,teamName,number_of_goals,team_id_for
23,2000,Flames,197,20
52,2001,Flames,198,20
84,2002,Flames,183,20
108,2003,Flames,200,20
147,2005,Flames,217,20
158,2006,Flames,261,20
196,2007,Flames,225,20
222,2008,Flames,250,20
269,2009,Flames,211,20
279,2010,Flames,256,20


## Number of saves per game_id:

In [9]:
query = """
SELECT
	game_id, SUM(saves) AS total_saves
FROM
	game_goalie_stats
GROUP BY
	game_id
"""
df_saves = pd.read_sql(query, con)

df_saves.head()

,game_id,total_saves
0,2000020001,45
1,2000020002,63
2,2000020003,45
3,2000020004,53
4,2000020005,42


## Checking ways to get number of goals

To check if the total number of goals calculated via the `game` table and that calculated via the `game_goalie_stats` match, we can run a (somewhat weird) query:

In [10]:
query = """
SELECT
	gg.game_id, gg.total_goals,
	goalie_data.possible_goals,
	(gg.total_goals - goalie_data.possible_goals) AS diff
FROM (
	SELECT
		game_id,
		(away_goals + home_goals) AS total_goals
	FROM
		game
	) AS gg
INNER JOIN (
	SELECT
		game_id,
		SUM(shots - saves) AS possible_goals
	FROM
		game_goalie_stats
	GROUP BY
		game_id			
	) AS goalie_data
ON gg.game_id = goalie_data.game_id
"""
df = pd.read_sql(query, con)
df.head()

,game_id,total_goals,possible_goals,diff
0,2016020045,11,10,1
1,2017020812,7,7,0
2,2015020314,5,4,1
3,2015020849,3,3,0
4,2017020586,3,3,0


There are some differences between the ways we calculate number of goals. A better understanding of the statistics of `goalie_stats` is needed here!

# Team performance questions:

## Given a season, which team has scored the most number of goals.

In [11]:
query = """
SELECT
	SUBSTR(game_plays.game_id, 1, 4) AS season,
	team_info.teamName,
	COUNT(game_plays.event) AS number_of_goals,
	game_plays.team_id_for
FROM
	game_plays
INNER JOIN
	team_info
		ON game_plays.team_id_for = team_info.team_id
WHERE
	event = 'Goal'
GROUP BY
	season, game_plays.team_id_for
ORDER BY
	season, number_of_goals DESC
"""

df = pd.read_sql(query, con)
df.head()

,season,teamName,number_of_goals,team_id_for
0,2000,Devils,293,1
1,2000,Penguins,271,5
2,2000,Avalanche,263,21
3,2000,Senators,258,9
4,2000,Red Wings,253,17


One can filter by season to get the rank.

## Goals conceded per team per season

In [12]:
query = """
SELECT
	SUBSTR(game_plays.game_id, 1, 4) AS season,
	team_info.teamName,
	COUNT(game_plays.event) AS goals_conceded,
	game_plays.team_id_against
FROM
	game_plays
INNER JOIN
	team_info
		ON game_plays.team_id_against = team_info.team_id
WHERE
	event = 'Goal'
GROUP BY
	season, game_plays.team_id_against
ORDER BY
	season, goals_conceded DESC;
"""
df_teams_conceded = pd.read_sql(query, con)
df_teams_conceded.head()

,season,teamName,goals_conceded,team_id_against
0,2000,Rangers,283,3
1,2000,Thrashers,277,11
2,2000,Lightning,275,14
3,2000,Islanders,261,2
4,2000,Penguins,252,5


## Goal difference

Using the last two queries as subqueries joined on `season` and `teamName`, we can calculate the `goal_difference` (number of golas scored minus number of goals conceded) by season by team. We'll use CTEs for a better readability:

In [13]:
query = """
WITH pro AS(
    SELECT
        SUBSTR(game_plays.game_id, 1, 4) AS season,
        team_info.teamName,
        COUNT(game_plays.event) AS number_of_goals,
        game_plays.team_id_for     
    FROM
        game_plays
    INNER JOIN
        team_info
            ON game_plays.team_id_for = team_info.team_id
    WHERE
        event = 'Goal'
    GROUP BY
        season, game_plays.team_id_for
),

con AS(
    SELECT
        SUBSTR(game_plays.game_id, 1, 4) AS season,
        team_info.teamName,
        COUNT(game_plays.event) AS goals_conceded,
        game_plays.team_id_against    
    FROM
        game_plays
    INNER JOIN
        team_info
            ON game_plays.team_id_against = team_info.team_id
    WHERE
        event = 'Goal'
    GROUP BY
        season, game_plays.team_id_against
)

SELECT
    pro.season, pro.teamName,
    pro.number_of_goals AS goals_scored,
    con.goals_conceded AS goals_conceded,
    (pro.number_of_goals - con.goals_conceded) AS goal_difference
FROM
	pro
INNER JOIN
	con
ON
    (pro.season = con.season)
    AND
        (pro.teamName = con.teamName)
ORDER BY
    pro.season, goal_difference DESC
"""
df = pd.read_sql(query, con)
df.head()

,season,teamName,goals_scored,goals_conceded,goal_difference
0,2000,Devils,293,192,101
1,2000,Avalanche,263,188,75
2,2000,Senators,258,191,67
3,2000,Stars,237,178,59
4,2000,Blues,243,188,55


# Individual performance

## Get best scorers per season

We'll only look at scorers with more than 40 goals per season.

In [14]:
query = """
WITH players AS(
    SELECT
        SUBSTR(game_skater_stats.game_id, 1, 4) AS season,
        game_skater_stats.player_id,
        game_skater_stats.team_id,
        player_info.firstName,
        player_info.lastName,
        player_info.primaryPosition,
        SUM(game_skater_stats.goals) AS goals_scored
    FROM
        game_skater_stats
    INNER JOIN
        player_info
        ON
            game_skater_stats.player_id = player_info.player_id
    GROUP BY
        season, game_skater_stats.player_id
)

SELECT
    players.season,
    players.firstName,
    players.lastName,
    players.primaryPosition,
    team_info.teamName,
    players.goals_scored
FROM
	players
INNER JOIN
    team_info
    ON
    players.team_id = team_info.team_id
WHERE
    players.goals_scored > 40
ORDER BY
    players.season, players.goals_scored DESC;
"""

df = pd.read_sql(query, con)
df.head()

,season,firstName,lastName,primaryPosition,teamName,goals_scored
0,2000,Pavel,Bure,RW,Panthers,59
1,2000,Joe,Sakic,C,Avalanche,54
2,2000,Jaromir,Jagr,RW,Penguins,52
3,2000,Peter,Bondra,RW,Capitals,45
4,2000,Alex,Kovalev,RW,Penguins,44


## Best goalies based on shots saved per season

We'll keep only goalies with more than 1500 saved shots per season.

In [15]:
query = """
WITH players AS(
    SELECT
        SUBSTR(game_goalie_stats.game_id, 1, 4) AS season,
        game_goalie_stats.player_id,
        game_goalie_stats.team_id,
        player_info.firstName,
        player_info.lastName,
        SUM(game_goalie_stats.saves) AS shots_saved,
        SUM(game_goalie_stats.shots) AS shots_taken
    FROM
        game_goalie_stats
    INNER JOIN
        player_info
        ON
            game_goalie_stats.player_id = player_info.player_id
    GROUP BY
        season, game_goalie_stats.player_id
)

SELECT
    players.season,
    players.firstName,
    players.lastName,
    team_info.teamName,
    players.shots_saved,
    players.shots_taken,
    CAST(players.shots_saved AS REAL) / CAST(players.shots_taken AS REAL) AS season_savePercentage
FROM players
INNER JOIN
    team_info
    ON
    players.team_id = team_info.team_id
WHERE
    players.shots_saved > 1500
ORDER BY
    players.season, players.shots_saved DESC;
"""
df = pd.read_sql(query, con)
df.head()

,season,firstName,lastName,teamName,shots_saved,shots_taken,season_savePercentage
0,2000,Arturs,Irbe,Hurricanes,1767,1947,0.907550
1,2000,Olie,Kolzig,Capitals,1764,1941,0.908810
2,2000,Curtis,Joseph,Maple Leafs,1744,1907,0.914525
3,2000,Tommy,Salo,Oilers,1677,1856,0.903556
4,2000,Sean,Burke,Coyotes,1629,1767,0.921902


Similar to the above, but ordering by `season_savePercentage` among goalies with at least 1000 shots taken (goal or not):

In [16]:
query = """
WITH players AS(
    SELECT
        SUBSTR(game_goalie_stats.game_id, 1, 4) AS season,
        game_goalie_stats.player_id,
        game_goalie_stats.team_id,
        player_info.firstName,
        player_info.lastName,
        SUM(game_goalie_stats.saves) AS shots_saved,
        SUM(game_goalie_stats.shots) AS shots_taken
    FROM
        game_goalie_stats
    INNER JOIN
        player_info
        ON
            game_goalie_stats.player_id = player_info.player_id
    GROUP BY
        season, game_goalie_stats.player_id
)

SELECT
    players.season,
    players.firstName,
    players.lastName,
    team_info.teamName,
    players.shots_saved,
    players.shots_taken,
    CAST(players.shots_saved AS REAL) / CAST(players.shots_taken AS REAL) AS season_savePercentage
FROM
	players
INNER JOIN
    team_info
    ON
    players.team_id = team_info.team_id
WHERE
    players.shots_taken > 1000
ORDER BY
    players.season, season_savePercentage DESC
"""
df = pd.read_sql(query, con)
df.head()

,season,firstName,lastName,teamName,shots_saved,shots_taken,season_savePercentage
0,2000,Mike,Dunham,Predators,1274,1381,0.922520
1,2000,Sean,Burke,Coyotes,1629,1767,0.921902
2,2000,Roman,Cechmanek,Flyers,1349,1464,0.921448
3,2000,Dominik,Hasek,Sabres,1589,1726,0.920626
4,2000,Manny,Fernandez,Wild,1055,1147,0.919791


# Getting data on "attempted shots"

In [17]:
query = """
SELECT
	SUBSTR(game_id, 1, 4) AS season,
	play_id, game_id, team_id_for, team_id_against,
	event,
	x, y,
	period, periodType, periodTime, periodTimeRemaining,
	st_x, st_y,
	(periodTime + periodTimeRemaining) AS time_sum
FROM
	game_plays
WHERE
	event IN ('Goal', 'Shot', 'Missed Shot', 'Blocked Shot')
ORDER BY
	season, play_id
"""
df = pd.read_sql(query, con)
df.head()

,season,play_id,game_id,team_id_for,team_id_against,event,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2000,2000020001_10,2000020001,21,25,Goal,NA,NA,2,REGULAR,120,NA,NA,NA,120
1,2000,2000020001_16,2000020001,21,25,Goal,NA,NA,2,REGULAR,1111,NA,NA,NA,1111
2,2000,2000020001_4,2000020001,25,21,Goal,NA,NA,1,REGULAR,760,NA,NA,NA,760
3,2000,2000020001_9,2000020001,25,21,Goal,NA,NA,1,REGULAR,1199,NA,NA,NA,1199
4,2000,2000020002_12,2000020002,6,9,Goal,NA,NA,2,REGULAR,722,NA,NA,NA,722


Counting occurance of each event:

In [18]:
query = """
SELECT
	event, COUNT(event)
FROM
	game_plays
WHERE
	event IN ('Goal', 'Shot', 'Missed Shot', 'Blocked Shot')
GROUP BY
	event;
"""
df = pd.read_sql(query, con)
df

,event,COUNT(event)
0,Blocked Shot,358851
1,Goal,133345
2,Missed Shot,296389
3,Shot,698365


## Selecting rows with non-null values for `x` and `y`.

In [19]:
query = """
SELECT
	SUBSTR(game_id, 1, 4) AS season,
	play_id, game_id, team_id_for, team_id_against,
	event,
	x, y,
	period, periodType, periodTime, periodTimeRemaining,
	st_x, st_y,
	(periodTime + periodTimeRemaining) AS time_sum
FROM
	game_plays
WHERE
	event IN ('Goal', 'Shot', 'Missed Shot', 'Blocked Shot')
	AND
		(x <> 'NA' AND y <> 'NA')
ORDER BY
	season, play_id
"""
df = pd.read_sql(query, con)
df.head()

,season,play_id,game_id,team_id_for,team_id_against,event,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2010,2010020001_10,2010020001,8,10,Shot,81,-7,1,REGULAR,86,1114,81,-7,1200
1,2010,2010020001_102,2010020001,8,10,Shot,45,-23,1,REGULAR,1092,108,45,-23,1200
2,2010,2010020001_104,2010020001,10,8,Missed Shot,-37,21,1,REGULAR,1120,80,37,-21,1200
3,2010,2010020001_107,2010020001,10,8,Shot,-48,30,1,REGULAR,1151,49,48,-30,1200
4,2010,2010020001_108,2010020001,10,8,Missed Shot,-1,-1,1,REGULAR,1186,14,1,1,1200


NOTE: The event `Blocked Shot` refers to
> A blocked shot occurs when an opponent's shot attempt is blocked by a skater, with his stick or body.

So, in our dataset, `team_id_for` refers to the defending team in this event.

## Goals by player

Get goals positions and game info plus the player_id of the socrer.

In [20]:
query = """
SELECT
	game_plays.play_id, game_plays.game_id,
	game_plays_players.player_id,
	game_plays.team_id_for, game_plays.team_id_against,
	game_plays.event, game_plays.secondaryType,
	game_plays.st_x, game_plays.st_y
FROM
	game_plays
INNER JOIN
	game_plays_players
	ON
	game_plays.play_id = game_plays_players.play_id
WHERE
	game_plays.event == 'Goal'
"""
df = pd.read_sql(query, con)
df.head()

,play_id,game_id,player_id,team_id_for,team_id_against,event,secondaryType,st_x,st_y
0,2016020045_6,2016020045,8470607,16,4,Goal,Wrap-around,88,-5
1,2016020045_6,2016020045,8473573,16,4,Goal,Wrap-around,88,-5
2,2016020045_6,2016020045,8473607,16,4,Goal,Wrap-around,88,-5
3,2016020045_6,2016020045,8474141,16,4,Goal,Wrap-around,88,-5
4,2016020045_97,2016020045,8473607,16,4,Goal,Backhand,76,-1


Using these queries we'll continue our data exploration with some visuals in the notebook [EDA](EDA.ipynb)

To help is organize our data later on, let's create two helper dataframes:
* A simple look up table for team_id and team_name
* A dataframe to relate game_id to the actual game date, home team and away team.

In [21]:
# team id to team name
query = """
SELECT
	team_id, teamName
FROM
	team_info
ORDER BY
	team_id ASC;
"""
df_teams = pd.read_sql(query, con)

In [22]:
df_teams

,team_id,teamName
0,1,Devils
1,2,Islanders
2,3,Rangers
3,4,Flyers
4,5,Penguins
5,6,Bruins
6,7,Sabres
7,8,Canadiens
8,9,Senators
9,10,Maple Leafs


In [23]:
# game_id info
query = """
SELECT
	game_id, date_time_GMT, home_team_id, away_team_id
FROM
	game;
"""
df_game_id = pd.read_sql(query, con)

In [24]:
df_game_id.head()

,game_id,date_time_GMT,home_team_id,away_team_id
0,2016020045,2016-10-19T00:30:00Z,16,4
1,2017020812,2018-02-07T00:00:00Z,7,24
2,2015020314,2015-11-24T01:00:00Z,52,21
3,2015020849,2016-02-17T00:00:00Z,12,52
4,2017020586,2017-12-30T03:00:00Z,24,20


In [25]:
df_game_id.tail()

,game_id,date_time_GMT,home_team_id,away_team_id
23730,2018030413,2019-06-02T00:00:00Z,19,6
23731,2018030414,2019-06-04T00:00:00Z,19,6
23732,2018030415,2019-06-07T00:00:00Z,6,19
23733,2018030416,2019-06-10T00:00:00Z,19,6
23734,2018030417,2019-06-13T00:00:00Z,6,19


Finally, let's dump these two into serialized objects for future use.

In [26]:
df_teams.to_pickle("df_teams.data")

In [27]:
df_game_id.to_pickle("df_games_id.data")

In [28]:
# Closing connection
con.close()